# Missing Data

데이터에 null값이 포함되어 있을 경우,대부분의 머신러닝 라이브러리에서 에러발생
따라서, missing data 전처리 필요함

1) drop columns : null값 포함한 컬럼 제외하기

2) Imputation

3) An Extension To Imputation

*Kaggle Machine learning Courses

In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [3]:
data=pd.read_csv("../data/melb_data.csv")

In [7]:
data.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

In [11]:
data.dropna(axis=0, subset=['Price'],inplace=True)
y = data['Price']
X=data.drop(['Price'],axis=1)
X = X.select_dtypes(exclude=['object'])

In [12]:
X.columns

Index(['Rooms', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude',
       'Propertycount'],
      dtype='object')

In [14]:
X_train, X_valid, y_train, y_valid=train_test_split(X,y,train_size=0.8,test_size=0.2,random_state=0)

In [15]:
X_train.head()

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
12167,1,5.0,3182.0,1.0,1.0,1.0,0.0,NaN,1940.0,-37.85984,144.9867,13240.0
6524,2,8.0,3016.0,2.0,2.0,1.0,193.0,NaN,NaN,-37.85800,144.9005,6380.0
8413,3,12.6,3020.0,3.0,1.0,1.0,555.0,NaN,NaN,-37.79880,144.8220,3755.0
2919,3,13.0,3046.0,3.0,1.0,1.0,265.0,NaN,1995.0,-37.70830,144.9158,8870.0
6043,3,13.3,3020.0,3.0,1.0,2.0,673.0,673.0,1970.0,-37.76230,144.8272,4217.0


In [16]:
X_train.shape

(10864, 12)

In [19]:
missing_val= X_train.isnull().sum()

In [29]:
# missing data 있는 컬럼 확인
missing_val[missing_val > 0]

Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64

In [30]:
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100,random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid,preds)

## 1. Drop Columns

In [38]:
# null값 포함한 col 추출
cols_with_missing = [col for col in X_train.columns 
                     if X_train[col].isnull().any()]

In [39]:
cols_with_missing

['Car', 'BuildingArea', 'YearBuilt']

In [44]:
# null값 포함한 컬럼 drop
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

In [41]:
reduced_X_train.columns

Index(['Rooms', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Landsize',
       'Lattitude', 'Longtitude', 'Propertycount'],
      dtype='object')

In [46]:
# null값 제거 후 정상적으로 돌아감 
score_dataset(reduced_X_train,reduced_X_valid,y_train,y_valid)

175703.48185157913

In [47]:
# null값 포함할 경우 에러 발생
score_dataset(X_train,X_valid,y_train,y_valid)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

## 2. Imputation

missing value 를 각 컬럼의 mean값으로 대체

In [48]:
from sklearn.impute import SimpleImputer

In [50]:
my_imputer = SimpleImputer()

In [60]:
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.fit_transform(X_valid))

In [55]:
imputed_X_train.isnull().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
dtype: int64

In [59]:
# column drop 했을때보다 error 값 작아짐
score_dataset(imputed_X_train,imputed_X_valid, y_train, y_valid)

169712.29865462164

## 3. An Extension to Imputation

기존 컬럼 유지하고, imputation 하는 컬럼 추가해서 계산

In [62]:
x_train_plus = X_train.copy()
x_valid_plus = X_valid.copy()

In [70]:
# 널 값 가진 컬럼 추가 생성하고, 널값인지 아닌지 True, False 값 넣어주기
for col in cols_with_missing:
    x_train_plus[col+'_was_missing'] = x_train_plus[col].isnull()
    x_valid_plus[col+'_was_missing'] = x_valid_plus[col].isnull()

In [71]:
x_train_plus[['Car','BuildingArea','YearBuilt','Car_was_missing','BuildingArea_was_missing','YearBuilt_was_missing']]

,Car,BuildingArea,YearBuilt,Car_was_missing,BuildingArea_was_missing,YearBuilt_was_missing
12167,1.0,NaN,1940.0,False,True,False
6524,1.0,NaN,NaN,False,True,True
8413,1.0,NaN,NaN,False,True,True
2919,1.0,NaN,1995.0,False,True,False
6043,2.0,673.0,1970.0,False,False,False
...,...,...,...,...,...,...
13123,2.0,NaN,NaN,False,True,True
3264,1.0,101.0,1950.0,False,False,False
9845,2.0,255.0,2002.0,False,False,False
10799,1.0,NaN,NaN,False,True,True


In [72]:
my_imputer = SimpleImputer()

In [74]:
#Impute
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(x_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.fit_transform(x_valid_plus))

In [75]:
imputed_X_train_plus.columns = x_train_plus.columns
imputed_X_valid_plus.columns = x_valid_plus.columns

In [76]:
score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid)

169763.6808357879

## 왜 Imputation 했을 때 결과가 더 좋은가?

null값을 포함한 컬럼 전체를 날려버릴경우, 사용할 수 있는 정보를 잃는것과 같다.
imputation 할 경우, 컬럼 전체를 날리기보다 null값을 보정해주면서 실제 데이터 활용하기때문에 더 성능이 좋다.

두가지를 같이 사용해서 null값이 너무 많이 포함된 columns은 날려버리고 (정보성 떨어짐), null값이 소수이며, feature로 유용한 컬럼일 경우에는 imputation 하면 성능을 올릴 수 있을듯!
